In [1]:
class Quotient_Ring:
    def __init__(self,n,p,N,d):
        ff.<gen>=GF(p,modulus="primitive") # coeficient field
        self.gen=gen
        self.ff=ff
        assert N%n==0, "{}|{} is false".format(n,N)
        self.p=p
        self.N=N
        self.n=n
        self.d=d
    def __str__(self):
        return "Quotient Ring: ({}) / (x^{}-(w_{})^{})".format(self.ff,self.n,self.N,self.d)
    def denominator(self):
        pr=self.ff["x"]
        x=pr("x")
        return x^self.n-E(self.N)^(self.d)
    def root(self,degree):
        assert (self.p-1)%degree==0, (self.p,degree)
        return self.gen^((self.p-1)/degree)

In [2]:
class Reduction:
    def __init__(self,qr):
        self.qr=qr
    def apply(self,a):
        pass
    def undo(self,A):
        pass
    def signature(self):
        pass
    
class Phi(Reduction):
    def __init__(self,qr,k):
        assert qr.n%k==0
        Reduction.__init__(self,qr)
        self.k=k
    def signature(self):
        n=self.qr.n//self.k
        return [Quotient_Ring(n,self.qr.p,self.qr.N,(self.qr.d+i*self.qr.N)//self.k) for i in range(self.k)]
    def apply(self,a):
        assert len(a)==self.qr.n
        m=self.qr.n//self.k
        A=[[0 for j in range(m)] for i in range(self.k)]
        w=self.qr.root(self.qr.N)
        for z in range(self.k):
            for i in range(self.k):
                for j in range(m):
                    A[z][j]+=a[i*m+j]*w^((self.qr.d+z*self.qr.N)*i//self.k)
        return A
    def undo(self,A):
        m=self.qr.n//self.k
        assert len(A)==self.k, (len(A),self.k)
        assert len(A[0])==m, (len(A[0]),self.k)
        a=[0 for i in range(self.qr.n)]
        w=self.qr.root(self.qr.N)
        for i in range(self.k):
            for j in range(m):
                for z in range(self.k):
                    el=A[z][j]*w^(-1*(self.qr.d//self.k+z*self.qr.N//self.k)*i)
                    #print((i,j,z),el)
                    a[i*m+j]+=el
                a[i*m+j]/=self.k
        return a
    def operations(self):
        return self.k*self.qr.n
        
class Gamma(Reduction):
    def __init__(self,qr):
        assert (qr.N-qr.d)%qr.n==0
        Reduction.__init__(self,qr)
    def signature(self):
        return Quotient_Ring(self.qr.n,self.qr.p,self.qr.N,self.qr.N)
    def apply(self,a):
        assert len(a)==self.qr.n
        assert (self.qr.N-self.qr.d)%self.qr.n==0
        zeta=qr.root(self.qr.N)^((self.qr.N-self.qr.d)/self.qr.n)
        return [a[i]*zeta^(i) for i in range(self.qr.n)]
    def undo(self,a):
        assert len(a)==self.qr.n
        assert (self.qr.N-self.qr.d)%self.qr.n==0
        zeta=qr.root(self.qr.N)^((self.qr.N-self.qr.d)/self.qr.n)
        return [a[i]*zeta^(self.qr.N-i) for i in range(self.qr.n)]
    def operations(self):
        return self.qr.n
    
class CT(Reduction):
    def __init__(self,qr,k):
        Reduction.__init__(self,qr)
        self.k=k
        self.phi=Phi(qr,k)
    def apply(self,a):
        return self.phi.apply(a)
    def undo(self,a):
        return self.phi.undo(a)
    def signature(self):
        return self.phi.signature()
    def operations(self):
        return self.phi.operations()
    
class GS(Reduction):
    def __init__(self,qr,k):
        Reduction.__init__(self,qr)
        self.k=k
        self.phi=Phi(qr,k)
        self.gamma=[Gamma(ps) for ps in self.phi.signature()]
    def apply(self,a):
        A=self.phi.apply(a)
        for i in range(self.k):
            A[i]=self.gamma[i].apply(A[i])
        return A
    def undo(self,gA):
        A=[self.gamma[i].undo(gA[i]) for i in range(self.k)]
        return self.phi.undo(A)
    def signature(self):
        return [g.signature() for g in self.gamma]
    def operations(self):
        return self.phi.operations()+sum(gamma.operations in self.gamma)
    
class Split(Reduction):
    def __init__(self,qr,k1,k2):
        Reduction.__init__(self,qr)
        self.k1=k1
        self.k2=k2
        self.phi1=Phi(qr,k1)
        self.phi2=[Phi(s,k2) for s in self.phi1.signature()]
        self.gamma=[[Gamma(s) for s in phi.signature()] for phi in self.phi2]
    def apply(self,a):
        A=self.phi1.apply(a)
        for i in range(self.k1):
            A[i]=self.phi2[i].apply(A[i])
            for j in range(self.k2):
                A[i][j]=self.gamma[i][j].apply(A[i][j])
        return A
    def undo(self,A):
        A=[self.phi2[i].undo([self.gamma[i][j].undo(A[i][j]) for j in range(self.k2)]) for i in range(self.k1)]
        return self.phi1.undo(A)
    def signature(self):
        return [[self.gamma[i][j].signature() for j in range(self.k2)] for i in range(self.k1)]
    def operations(self):
        return self.phi1.operations()+sum(phi.operations() for phi in self.phi2)+sum(g.operations() for g in gamma for gamma in self.gamma)
    

In [4]:
n=6
N=6
p=13
k=2

qr=Quotient_Ring(n,p,N,N)
print(qr)

phi=Phi(qr,k)
for s in phi.signature():
    print(s)
    
a=[GF(p)(i) for i in range(n)]

print(a)
A=phi.apply(a)
print(A)

ap=phi.undo(A)
print(ap)

b=a[0:n//k]
qr2=phi.signature()[0]
gamma=Gamma(qr2)
print(gamma.signature())
print(gamma.apply(b))
print(gamma.undo(gamma.apply(b)))

Quotient Ring: (Finite Field of size 13) / (x^6-(w_6)^6)
Quotient Ring: (Finite Field of size 13) / (x^3-(w_6)^3)
Quotient Ring: (Finite Field of size 13) / (x^3-(w_6)^6)
[0, 1, 2, 3, 4, 5]
[[10, 10, 10], [3, 5, 7]]
[0, 1, 2, 3, 4, 5]
Quotient Ring: (Finite Field of size 13) / (x^3-(w_6)^6)
[0, 4, 6]
[0, 1, 2]


In [84]:
n=16
N=n
p=17
k=4
k2=2

qr=Quotient_Ring(n,p,N,N)
print(qr)

#a=[GF(p)((i+6)^4) for i in range(n)]
a=[i+1 for i in range(n)]
print("a",a)

print("\nCooley-Tukey")
ct=CT(qr,k)
for s in ct.signature():
    print(s)
print("A",ct.apply(a))
print("a",ct.undo(ct.apply(a)))

print("\nGentleman-Sande")
gs=GS(qr,k)
for s in gs.signature():
    print(s)
print("A",gs.apply(a))
print("a",gs.undo(gs.apply(a)))


Quotient Ring: (Finite Field of size 17) / (x^16-(w_16)^16)
a [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

Cooley-Tukey
Quotient Ring: (Finite Field of size 17) / (x^4-(w_16)^4)
Quotient Ring: (Finite Field of size 17) / (x^4-(w_16)^8)
Quotient Ring: (Finite Field of size 17) / (x^4-(w_16)^12)
Quotient Ring: (Finite Field of size 17) / (x^4-(w_16)^16)
A [[7, 7, 7, 7], [9, 9, 9, 9], [11, 11, 11, 11], [11, 15, 2, 6]]
a [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

Gentleman-Sande
Quotient Ring: (Finite Field of size 17) / (x^4-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^4-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^4-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^4-(w_16)^16)
A [[7, 2, 3, 13], [9, 13, 15, 16], [11, 16, 14, 8], [11, 15, 2, 6]]
a [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
split
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
Quotient

In [95]:
n=16
N=n
p=17
k=4
k2=2

qr=Quotient_Ring(n,p,N,N)
print(qr)

#a=[GF(p)((i+6)^4) for i in range(n)]
a=[i+1 for i in range(n)]
print("a",a)

print("split")
split=Split(qr,k,k2)
for s in split.signature():
    for s2 in s:
        print(s2)
print("A",split.apply(a))
print("a",split.undo(split.apply(a)))

Quotient Ring: (Finite Field of size 17) / (x^16-(w_16)^16)
a [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
split
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
Quotient Ring: (Finite Field of size 17) / (x^2-(w_16)^16)
A [[[2, 5], [12, 1]], [[7, 3], [11, 14]], [[6, 13], [16, 14]], [[9, 15], [13, 4]]]
a [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
